In [1]:
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
from alibi.datasets import fetch_adult
from alibi.explainers import CounterFactualProto
import pandas as pd 
import tensorflow as tf
from alibi.explainers import CounterFactual
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from ceteris_paribus.profiles import individual_variable_profile
from ceteris_paribus.plots.plots import plot_notebook, plot
from ceteris_paribus.explainer import explain
import imgkit
import shutil
import base64
from ceml.sklearn import generate_counterfactual

In [2]:
data = pd.read_csv('titanic.csv')
data.rename(columns={'Survived': 'class'}, inplace=True)
data['Sex'] = data['Sex'].map({'male':'Male','female':'Female'})
data['Embarked'] = data['Embarked'].map({'S':'Southampton','C':'Cherbourg','Q':'Queenstown'})
data['Pclass'] = data['Pclass'].map({1:'First', 2:'Second', 3:'Third'})
data['Relatives'] = data['SibSp'] + data['Parch']

data = data.drop(['PassengerId', 'Name','Ticket','Cabin', 'SibSp', 'Parch'], axis=1)
data = data.dropna()

f = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Relatives']

features = data.drop('class', axis=1)
#print(features)

training_features, testing_features, training_target, testing_target = \
    train_test_split(features, data['class'].values, random_state=None)

In [3]:
numeric_features = ['Age', 'Fare', 'Relatives']
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())])

categorical_features = ['Pclass', 'Sex', 'Embarked']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [5]:
model = Pipeline(steps=[('preprocessor', preprocessor),
('classifier', XGBClassifier())])

In [6]:
model.fit(training_features, training_target)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   MinMaxScaler(copy=True,
                                                                                feature_range=(0,
                                                                                               1)))],
                                                           verbose=False),
                                                  ['Age', 'Fare', 'Relatives']),
                                                 ('cat',
                                                  P

In [7]:
print("XGB {}".format(accuracy_score(testing_target, model.predict(testing_features))))

XGB 0.7921348314606742


In [17]:
test = (model['preprocessor'].transform(pd.DataFrame([{'Pclass': 'First', 'Sex': 'Female', 'Age': 25.0, 'Fare': 0.0000, 'Embarked': 'Southampton', 'Relatives': 0}])))
print(test)

[[0.30599369 0.         0.         1.         0.         0.
  1.         0.         0.         0.         1.        ]]


In [19]:
print(generate_counterfactual(model['classifier'], test, y_target=1))


ValueError: <class 'xgboost.sklearn.XGBClassifier'> is not supported